In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import pinecone
from openai import OpenAI

In [ ]:
pc=pinecone.Pinecone(api_key="60a0f303-f99f-4655-b726-fee96f255569")
pc.create_index(name="rag", dimension=1536, metric="cosine", spec=pinecone.ServerlessSpec(cloud="aws", region="us-east-1"))


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']


In [ ]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'], 
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({"values": embedding, 
                           "id": review['professor'], 
                           "metadata": {
                               "review": review["review"], 
                               "subject": review['subject'],
                               "stars" : review["stars"]}})


In [ ]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)


In [ ]:
#Index stats
index.describe_index_stats()